# El problema SAT

El *Boolean SATisfiability problem* es el problema de determinar si existe una interpretación que satisfaga una fórmula booleana dada. Si sí, la fórmula es *satisfacible*, si no es *insatisfacible*. Por ejemplo, `(a ∨ b) ∧ (¬a)` es satisfacible pero `(a ∨ b) ∧ (¬a) ∧ (¬b)` no lo es.

SAT fue el primer problema que se demostró ser NP-complete (Teorema de Cook–Levin). Sin embargo, en la práctica se pueden resolver problemas SAT con miles de fórmulas y millones de variables.

Una *fórmula booleana* es una expresión formada por variables y los operadores AND (conjunción, ∧), OR (disjuncióin, ∨), NOT (negación, ¬), y paréntesis.

Un *literal* es una variable o su negación (NOT). Una *cláusula* es una disjunción (ORs) de literales. Una *fórmula normal conjuntiva* (CNF) es una conjunción (ANDs) de cláusulas. Por ejemplo, `(a ∨ b) ∧ (¬a)` es una CNF pero `((a ∨ b) ∧ (¬a ∨ c)) ∨ (¬c ∨ b)` no lo es. Cualquier fórmula se puede expresar como una CNF.

Si restringimos el problema al caso en que cada cláusula tiene $k$ literales, se llama *$k$-SAT*. El problema $2$-SAT se puede resolver en timpo polinomial pero $3$-SAT es NP-completo. Cualquier fórmula es equivalente a una fórmula de $3$-SAT aunque tal vez tengamos que incrementar el número de variables.

## SAT-solvers

Aquí hay una lista de programas que lo hacen.

**CryptoMiniSat**

https://github.com/msoos/cryptominisat

**MiniSat**

http://minisat.se/

Este es probablemente el más popular de todos.

**PySAT**

https://pypi.org/project/python-sat/

Contiene varios algoritmos: CaDiCaL, Glucose, Lingeling, MapleLCMDistChronoBT, MapleCM, Maplesat, Minicard, Minisat.


## Formato

Hay un formato más o menos estándar que usan los SAT-solvers como entrada.

Hay que crear un archvo, éste puede tener varios renglones. Los renglones que empiezan con `c` son comentarios y se ignoran.

El primer renglón (no-comentario) debe ser de esta forma:

`p cnf NUMBER_OF_VARIABLES NUMBER_OF_CLAUSES`

Cada renglón siguiente (no-comentario) define una cláusula. Estos renglones tienen números enteros separados por un espacio. Un entero positivo $k$ representa la variable $x_k$, mientras que $-k$ representa $¬ x_k$. Cada renglón debe terminar con el número $0$.

### Ejemplo

Para la fórmula
```
(x1 ∨ ¬x5 ∨ x4) ∧
(¬x1 ∨ x5 ∨ x3 ∨ x4) ∧
(¬x3 ∨ x4)
```
podemos crear un archivo con este texto:
```
c Mi primer problema SAT
p cnf 5 3
1 -5 4 0
-1 5 3 4 0
-3 4 0
``` 

La fórmula
```
(x1) ∧
(¬x2) ∧
(¬x3) ∧
(¬x1 ∨ x2 ∨ x3)
```
corresponde a:
```
p cnf 3 4
1 0
-2 0
-3 0
-1 2 3 0
```

In [13]:
with open("sat1.txt", "w") as f: #w para abrir el archivo en modo write
    f.write("c Mi primer problema SAT\n")
    f.write("p cnf 5 3\n")
    f.write("1 -5 4 0\n")
    f.write("-1 5 3 4 0\n")
    f.write("-3 4 0\n")

El programa anterior crea un archivo con el 1er ejemplo. Lo podemos correr desde una terminal mediante un comando similar a

`cryptominisat5-win-amd64-nogauss sat1.txt`

dependiendo de el SAT-solver que usen y su sistema operativo. En caso de haber problemas, pueden usar este link para correr las cosas en linea: https://msoos.github.io/cryptominisat_web/

Después de correr este comando deberían poder ver (tal vez después de unas estadísticas) la respuesta

```
s SATISFIABLE
v -1 -2 -3 -4 -5 0
```

Esto significa que la fórmula sí es satisfacible, se satisface con los valores $x_i=FALSE$ para $i=1,\dots, 5$. Puede haber otros valores para los cuáles la fórmula se satisface. Para confirmar si es único podemos cambiar el archivo.

In [14]:
with open("sat1.txt", "a") as f: #a para agregar al archivo sin borrar lo anterior
    f.write("1 2 3 4 5 0\n")

Esta vez obtenemos la respuesta

```
s SATISFIABLE
v -1 2 -3 -4 -5 0
```

que nos da la solución $x_2=TRUE$, $x_i=FALSE$ para $i=1,3,4,5$.

In [15]:
with open("sat2.txt", "w") as f:
    f.write("p cnf 3 4\n1 0\n-2 0\n-3 0\n-1 2 3 0\n")

Este programa nos da

```
s UNSATISFIABLE
```

que significa que la fórmula no se puede satisfacer. Además es pósible extraer una prueba formar de que este es el caso, pero no veremos eso en este curso.

## Ramsey

Imaginemos que queremos ver si las aristas de una gráfica se pueden colorear con dos colores sin formar triángulos monocromáticos.

Podemos crear una variable para cada arista y buscamos asignar TRUE o FALSE a cada variable de forma que no tengamos un triángulo cuyas variables sean todas TRUE ni todas FALSE.

Si $a,b,c$ son variables que forman un triángulo, esto lo podemos escribir como
```
(a ∨ b ∨ c) ∧
(¬a ∨ ¬b ∨ ¬c)
```
Así garantizamos que haya un TRUE y un FALSE.

Hagamos esto para la gráfica completa de $n$ vértices. Los casos interesantes son $n=5,6$.

In [20]:
n=10

listaDeAristas = [(i,j) for i in range(n-1) for j in range(i+1,n)]
listaDeTriangulos = []
for i in range(n-2):
    for j in range(i+1,n-1):
        for k in range(j+1,n):
            a=listaDeAristas.index((i,j))+1
            b=listaDeAristas.index((i,k))+1
            c=listaDeAristas.index((j,k))+1
            listaDeTriangulos.append((a,b,c))
numVars = len(listaDeAristas)
numClaus = 2*len(listaDeTriangulos)
with open("ramsey.txt", "w") as f:
    f.write(f"c Ramsey para K_{n}\n")
    f.write(f"p cnf {numVars} {numClaus}\n")
    for a,b,c in listaDeTriangulos:
        f.write(f"{a} {b} {c} 0\n")
        f.write(f"{-a} {-b} {-c} 0\n")

In [19]:
listaDeAristas

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 4),
 (3, 5),
 (4, 5)]

### Problemas

1. Se necesita establecer un horario para $4$ cursos que se darán durante $3$ días con las siguientes condiciones: 
    - Cada día está dividido en $3$ periodos.
    - Durante cada perdiodo se debe dar exactamente un curso.
    - No se debe dar el mismo curso dos veces el mismo día.
    - Cada curso requiere de al menos dos periodos.

    Haz un programa que genere un horario válido.

2. Ver si con 3 colores se pueden colorear los vértices de la gráfica de Petersen de forma que no hayan dos vértices adyacentes del mismo color.

La gráfica de Petersen tiene como vértices las parejas $\{i,j\}$ con $i\neq j$ y dos vértices son adyacentes si son disjuntos.

Sugerencia: por cada vértices poner 3 variables, cada variable de un vértice representa si se pinta de ese color o no.

3. Ver si los vértices de la gráfica completa en $n$ vértices se puede colorear con $k$ colores de forma que no hayan dos vértices del mismo color adyacentes.

4. Decidir si se puede terminar de resolver un Sudoku de $4\times 4$.

Sugerencia: Por cada cuadro poner 4 variables $v_1,\dots,v_4$. La variable $v_i$ de un cuadro dado dice si en ese cuadro se va a escribir el número $i$. Al final se pueden poner cláusulas extra que dicen qué cuadros ya estaban llenos desde el principio.

5. Camino Hamiltoniano

Sugerencia: Si tenemos n vértices, las variables serán $x_{i,j}$ con $1\le i,j\le n$. El valor $x_{i,j}=TRUE$ significa que en el $i$-ésimo vértice del camino estamos en el vértice $j$ de la gráfica.

Sugerencia para el 1.
Piensen en usar 4 variables por periodo, cada una corresponde a un curso para ese periodo x1,x2,x3,x4.

Esta cláusila garantiza que le asignemos al menos una curso a cada periodo:
```
x1 OR x2 OR x3 OR x4
```

Para evitar asignar 2 cursos a un periodo puedo hacer esto:
```
-x1 OR -x2
-x1 OR -x3
-x1 OR -x4
-x2 OR -x3
-x2 OR -x4
-x3 OR -x4
```

In [29]:
#Inicio de problema 1
dias = 3
periodos = 3
cursos = 4
minperiodos = 2

variables = [(dia,periodo,curso) for dia in range(dias) for periodo in range(periodos) for curso in range(cursos)]
clausulas = []

#Mala forma de hacerlo, usar funciones.
for dia in range(dias):
    for periodo in range(periodos):
        exact1 = []
        for curso in range(cursos):
            exact1.append(str(variables.index((dia,periodo,curso))+1))
        clausulas.append(" ".join(exact1)+" 0")
        for i in range(len(exact1)):
            for j in range(i+1, len(exact1)):
                clausulas.append(f"-{exact1[i]} -{exact1[j]} 0")
print(clausulas)

['1 2 3 4 0', '-1 -2 0', '-1 -3 0', '-1 -4 0', '-2 -3 0', '-2 -4 0', '-3 -4 0', '5 6 7 8 0', '-5 -6 0', '-5 -7 0', '-5 -8 0', '-6 -7 0', '-6 -8 0', '-7 -8 0', '9 10 11 12 0', '-9 -10 0', '-9 -11 0', '-9 -12 0', '-10 -11 0', '-10 -12 0', '-11 -12 0', '13 14 15 16 0', '-13 -14 0', '-13 -15 0', '-13 -16 0', '-14 -15 0', '-14 -16 0', '-15 -16 0', '17 18 19 20 0', '-17 -18 0', '-17 -19 0', '-17 -20 0', '-18 -19 0', '-18 -20 0', '-19 -20 0', '21 22 23 24 0', '-21 -22 0', '-21 -23 0', '-21 -24 0', '-22 -23 0', '-22 -24 0', '-23 -24 0', '25 26 27 28 0', '-25 -26 0', '-25 -27 0', '-25 -28 0', '-26 -27 0', '-26 -28 0', '-27 -28 0', '29 30 31 32 0', '-29 -30 0', '-29 -31 0', '-29 -32 0', '-30 -31 0', '-30 -32 0', '-31 -32 0', '33 34 35 36 0', '-33 -34 0', '-33 -35 0', '-33 -36 0', '-34 -35 0', '-34 -36 0', '-35 -36 0']
